In [1]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from flair.visual.training_curves import Plotter

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
# initialize embeddings. This takes time to load.
embedding_types: List[TokenEmbeddings] = [
    # GloVe embeddings for arabic
    WordEmbeddings('ar'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

2019-04-13 22:29:10,404 this function is deprecated, use smart_open.open instead


In [3]:

# 1. get the corpus
columns = {0: 'text', 1: 'ner'}
data_folder = '/home/jupyter/data_ner/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, 
  train_file='train_full.txt', 
  dev_file='dev.txt')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


# initialize sequence tagger
from flair.models import SequenceTagger

# have a relatively small hidden_size
tagger: SequenceTagger = SequenceTagger(hidden_size=64,
                                        dropout = 0.2,
                                        rnn_layers = 2,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)# initialize trainer
# 6. Initialize trainer
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

from flair.optim import AdamW
# optimizer = optimizer(momentum=0.9, dampening=0.9)
trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=AdamW)


2019-04-13 22:29:12,738 Reading data from /home/jupyter/data_ner
2019-04-13 22:29:12,739 Train: /home/jupyter/data_ner/train_full.txt
2019-04-13 22:29:12,740 Dev: /home/jupyter/data_ner/dev.txt
2019-04-13 22:29:12,741 Test: None


In [4]:
import os
# 7. start training
result_folder = '/home/jupyter/result_64_2_l01_w01_d02_adamw_full/'
trainer.train(result_folder,
              EvaluationMetric.MICRO_F1_SCORE,
              learning_rate=0.1,
              mini_batch_size=32,
              weight_decay = 0.1,
              max_epochs=200,
              checkpoint=False)


2019-04-13 22:29:15,477 ----------------------------------------------------------------------------------------------------
2019-04-13 22:29:15,478 Evaluation method: MICRO_F1_SCORE
2019-04-13 22:29:15,481 ----------------------------------------------------------------------------------------------------
2019-04-13 22:29:15,854 epoch 1 - iter 0/38 - loss 90.95499420
2019-04-13 22:29:16,776 epoch 1 - iter 3/38 - loss 59.69537354
2019-04-13 22:29:17,875 epoch 1 - iter 6/38 - loss 48.22286170
2019-04-13 22:29:18,880 epoch 1 - iter 9/38 - loss 38.85229969
2019-04-13 22:29:19,896 epoch 1 - iter 12/38 - loss 35.50772843
2019-04-13 22:29:20,669 epoch 1 - iter 15/38 - loss 31.82087958
2019-04-13 22:29:21,813 epoch 1 - iter 18/38 - loss 29.77491349
2019-04-13 22:29:22,721 epoch 1 - iter 21/38 - loss 28.01476834
2019-04-13 22:29:23,630 epoch 1 - iter 24/38 - loss 26.29737274
2019-04-13 22:29:24,748 epoch 1 - iter 27/38 - loss 25.08990628
2019-04-13 22:29:26,217 epoch 1 - iter 30/38 - loss 24.4

{'test_score': 0.8118,
 'dev_score_history': [0.0,
  0.0,
  0.0219,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0197,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0097,
  0.0198,
  0.0,
  0.0,
  0.0229,
  0.0102,
  0.0545,
  0.0439,
  0.0696,
  0.0253,
  0.1234,
  0.0458,
  0.013,
  0.0418,
  0.1491,
  0.2598,
  0.1285,
  0.0067,
  0.0133,
  0.2669,
  0.3494,
  0.282,
  0.2831,
  0.2444,
  0.1945,
  0.4339,
  0.4222,
  0.4072,
  0.4211,
  0.4193,
  0.3681,
  0.4715,
  0.4443,
  0.4491,
  0.4672,
  0.4807,
  0.5059,
  0.3041,
  0.458,
  0.4801,
  0.365,
  0.3828,
  0.4816,
  0.4617,
  0.4592,
  0.4979,
  0.4108,
  0.4514,
  0.4467,
  0.3698,
  0.5098,
  0.5445,
  0.4855,
  0.4995,
  0.5136,
  0.5076,
  0.5864,
  0.5666,
  0.6116,
  0.5814,
  0.581,
  0.5955,
  0.6125,
  0.5904,
  0.6123,
  0.5625,
  0.5969,
  0.604,
  0.656,
  0.646,
  0.643,
  0.6814,
  0.6717,
  0.6468,
  0.6586,
  0.666,
  0.6825,
  0.6655,
  0.6824,
  0.6767,
  0.6771,
  0.6889,
  0.6905,
  0.7039,
  0.701

In [9]:
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves(os.path.join(result_folder, 'loss.tsv'))
plotter.plot_weights(os.path.join(result_folder, 'weights.txt'))